## 1. Configuração Inicial

In [2]:
import warnings
import json
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from tqdm import tqdm
import time

# Importando funções do módulo local
from embedding_utils import (
    get_embedding,
    cosine_similarity,
    buscar_wikipedia,
    dividir_em_chunks,
)

warnings.filterwarnings("ignore")

# Carrega variáveis do arquivo .env
load_dotenv()

# Cliente OpenAI
client = OpenAI()

print("✅ Bibliotecas importadas e cliente OpenAI configurado!")

✅ Bibliotecas importadas e cliente OpenAI configurado!


## 2. Lista de 30 Escritores Brasileiros de Renome

Selecionamos escritores importantes da literatura brasileira de diferentes períodos.

In [20]:
# Lista de 50 escritores brasileiros clássicos (formato Wikipedia)
escritores_brasileiros = [
    # Romantismo e Realismo
    "Machado_de_Assis",
    "José_de_Alencar",
    "Lima_Barreto_(escritor)",
    "Aluísio_Azevedo",
    "Joaquim_Manuel_de_Macedo",
    "Visconde_de_Taunay",
    "Raul_Pompeia",
    "Bernardo_Guimarães",
    # Modernismo - 1ª Geração
    "Mário_de_Andrade",
    "Oswald_de_Andrade",
    "Manuel_Bandeira",
    "Menotti_Del_Picchia",
    "Cassiano_Ricardo",
    # Modernismo - 2ª Geração (Prosa)
    "Graciliano_Ramos",
    "Jorge_Amado",
    "José_Lins_do_Rego",
    "Érico_Veríssimo",
    "Rachel_de_Queiroz",
    "Dyonelio_Machado",
    # Modernismo - 2ª Geração (Poesia)
    "Carlos_Drummond_de_Andrade",
    "Cecília_Meireles",
    "Vinícius_de_Moraes",
    "Murilo_Mendes",
    "Jorge_de_Lima",
    # Modernismo - 3ª Geração
    "Guimarães_Rosa",
    "Clarice_Lispector",
    "João_Cabral_de_Melo_Neto",
    "Ferreira_Gullar",
    # Parnasianismo e Simbolismo
    "Olavo_Bilac",
    "Raimundo_Correia",
    "Gregório_de_Matos",
    "Cruz_e_Sousa",
    "Alphonsus_de_Guimaraens",
    # Pré-Modernismo
    "Euclides_da_Cunha",
    "Monteiro_Lobato",
    "Augusto_dos_Anjos",
    "Coelho_Neto",
    "Graça_Aranha",
    # Poesia Romântica
    "Castro_Alves",
    "Gonçalves_Dias",
    "Álvares_de_Azevedo",
    "Casimiro_de_Abreu",
    "Fagundes_Varela",
    # Contemporâneos Clássicos
    "Rubem_Braga",
    "Rubem_Fonseca",
    "Lygia_Fagundes_Telles",
    "Hilda_Hilst",
    "Ariano_Suassuna",
    "Autran_Dourado",
]

print(
    f"📚 Lista de {len(escritores_brasileiros)} escritores brasileiros clássicos para análise:"
)
for i, escritor in enumerate(escritores_brasileiros, 1):
    print(f"   {i:2d}. {escritor.replace('_', ' ')}")

📚 Lista de 49 escritores brasileiros clássicos para análise:
    1. Machado de Assis
    2. José de Alencar
    3. Lima Barreto (escritor)
    4. Aluísio Azevedo
    5. Joaquim Manuel de Macedo
    6. Visconde de Taunay
    7. Raul Pompeia
    8. Bernardo Guimarães
    9. Mário de Andrade
   10. Oswald de Andrade
   11. Manuel Bandeira
   12. Menotti Del Picchia
   13. Cassiano Ricardo
   14. Graciliano Ramos
   15. Jorge Amado
   16. José Lins do Rego
   17. Érico Veríssimo
   18. Rachel de Queiroz
   19. Dyonelio Machado
   20. Carlos Drummond de Andrade
   21. Cecília Meireles
   22. Vinícius de Moraes
   23. Murilo Mendes
   24. Jorge de Lima
   25. Guimarães Rosa
   26. Clarice Lispector
   27. João Cabral de Melo Neto
   28. Ferreira Gullar
   29. Olavo Bilac
   30. Raimundo Correia
   31. Gregório de Matos
   32. Cruz e Sousa
   33. Alphonsus de Guimaraens
   34. Euclides da Cunha
   35. Monteiro Lobato
   36. Augusto dos Anjos
   37. Coelho Neto
   38. Graça Aranha
   39. Castr

## 3. Coleta de Dados da Wikipedia

In [21]:
# Buscando conteúdo da Wikipedia para cada escritor
conteudos_escritores = {}
erros = []

print("🔍 Buscando informações na Wikipedia...\n")

for escritor in tqdm(escritores_brasileiros, desc="Coletando dados"):
    try:
        dados = buscar_wikipedia(escritor)
        conteudos_escritores[escritor] = dados
        time.sleep(0.5)  # Delay para não sobrecarregar a Wikipedia
    except Exception as e:
        erros.append((escritor, str(e)))
        print(f"   ⚠️ Erro ao buscar {escritor}: {e}")

print(f"\n✅ {len(conteudos_escritores)} páginas carregadas com sucesso!")
if erros:
    print(f"⚠️ {len(erros)} erros encontrados")

🔍 Buscando informações na Wikipedia...



Coletando dados: 100%|██████████| 49/49 [00:52<00:00,  1.08s/it]


✅ 49 páginas carregadas com sucesso!


In [24]:
# Visualizando um exemplo
if conteudos_escritores:
    exemplo_escritor = list(conteudos_escritores.keys())[0]
    exemplo_dados = conteudos_escritores[exemplo_escritor]

    print(f"📖 Exemplo - {exemplo_dados['titulo']}")
    print(f"🔗 URL: {exemplo_dados['url']}")
    print(f"📝 Número de parágrafos: {len(exemplo_dados['conteudo'])}")
    print(f"\n📄 Primeiro parágrafo:")
    print(exemplo_dados["conteudo"][0][:500] + "...")

📖 Exemplo - Machado de Assis
🔗 URL: https://pt.wikipedia.org/wiki/Machado_de_Assis
📝 Número de parágrafos: 107

📄 Primeiro parágrafo:
Joaquim Maria Machado de Assis (Rio de Janeiro, 21 de junho de 1839 – Rio de Janeiro, 29 de setembro de 1908) foi um escritor brasileiro, amplamente reconhecido por críticos, estudiosos, escritores e leitores como o maior expoente da literatura brasileira. Sua produção literária abrangeu praticamente todos os gêneros, incluindo poesia, romance, crônica, dramaturgia, conto, folhetim, jornalismo e crítica literária. Machado de Assis testemunhou a Abolição da Escravatura e a transição política do B...


## 4. Criação de Chunks e Geração de Embeddings

In [25]:
# Criando chunks de todos os conteúdos
todos_chunks = []
metadados_escritores = []

print("📦 Criando chunks para cada escritor...\n")

for escritor, dados in conteudos_escritores.items():
    # Junta todos os parágrafos
    texto_completo = " ".join(dados["conteudo"])

    # Debug: verificar se o texto está vazio
    if len(texto_completo.strip()) < 100:
        print(
            f"   ⚠️ {dados['titulo']}: texto muito curto ({len(texto_completo)} chars)"
        )
        print(f"      Parágrafos retornados: {len(dados['conteudo'])}")
        if dados["conteudo"]:
            print(f"      Primeiro parágrafo: {dados['conteudo'][0][:100]}...")
        continue

    # Divide em chunks
    chunks = dividir_em_chunks(
        texto_completo, titulo_pagina=dados["titulo"], tamanho_max=1000, overlap=150
    )

    print(f"   📄 {dados['titulo']}: {len(chunks)} chunks")

    for chunk in chunks:
        todos_chunks.append(chunk)
        metadados_escritores.append(
            {
                "escritor": dados["titulo"],
                "escritor_wiki": escritor,
                "url": dados["url"],
            }
        )

print(f"\n✅ Total: {len(todos_chunks)} chunks criados!")

# Verificar escritores sem chunks
escritores_com_chunks = set([m["escritor"] for m in metadados_escritores])
escritores_buscados = set([d["titulo"] for d in conteudos_escritores.values()])
sem_chunks = escritores_buscados - escritores_com_chunks
if sem_chunks:
    print(f"\n⚠️ Escritores sem chunks: {sem_chunks}")

📦 Criando chunks para cada escritor...

   📄 Machado de Assis: 133 chunks
   📄 José de Alencar: 40 chunks
   📄 Lima Barreto (escritor): 16 chunks
   📄 Aluísio Azevedo: 4 chunks
   📄 Joaquim Manuel de Macedo: 4 chunks
   📄 Visconde de Taunay: 11 chunks
   📄 Raul Pompeia: 15 chunks
   📄 Bernardo Guimarães: 6 chunks
   📄 Mário de Andrade: 34 chunks
   📄 Oswald de Andrade: 31 chunks
   📄 Manuel Bandeira: 16 chunks
   📄 Menotti Del Picchia: 6 chunks
   📄 Cassiano Ricardo: 5 chunks
   📄 Graciliano Ramos: 9 chunks
   📄 Jorge Amado: 19 chunks
   📄 José Lins do Rego: 15 chunks
   📄 Érico Veríssimo: 32 chunks
   📄 Rachel de Queiroz: 35 chunks
   📄 Dyonelio Machado: 5 chunks
   📄 Carlos Drummond de Andrade: 6 chunks
   📄 Cecília Meireles: 16 chunks
   📄 Vinícius de Moraes: 34 chunks
   📄 Murilo Mendes: 30 chunks
   📄 Jorge de Lima: 14 chunks
   📄 Guimarães Rosa: 9 chunks
   📄 Clarice Lispector: 35 chunks
   📄 João Cabral de Melo Neto: 8 chunks
   📄 Ferreira Gullar: 12 chunks
   📄 Olavo Bilac: 8 c

In [ ]:
# Gerando embeddings para todos os chunks
print("🧠 Gerando embeddings para todos os chunks...\n")
print("   ⏳ Isso pode levar alguns minutos...\n")

embeddings_escritores = []

for i, chunk in enumerate(tqdm(todos_chunks, desc="Gerando embeddings")):
    emb = get_embedding(client, chunk)
    embeddings_escritores.append(emb)

    # Pequena pausa para respeitar rate limits
    if (i + 1) % 50 == 0:
        time.sleep(1)

embeddings_escritores = np.array(embeddings_escritores)

print(f"\n✅ {len(embeddings_escritores)} embeddings gerados!")
print(f"📐 Dimensão dos embeddings: {embeddings_escritores.shape}")

🧠 Gerando embeddings para todos os chunks...

   ⏳ Isso pode levar alguns minutos...



Gerando embeddings:  63%|██████▎   | 612/972 [04:21<02:25,  2.48it/s]

In [ ]:
# Salvando os dados para uso futuro
np.save("embeddings_escritores.npy", embeddings_escritores)

with open("metadados_escritores.json", "w", encoding="utf-8") as f:
    json.dump(metadados_escritores, f, ensure_ascii=False, indent=2)

with open("chunks_escritores.json", "w", encoding="utf-8") as f:
    json.dump(todos_chunks, f, ensure_ascii=False, indent=2)

print("💾 Dados salvos:")
print("   - embeddings_escritores.npy")
print("   - metadados_escritores.json")
print("   - chunks_escritores.json")

## 5. Busca Semântica: Chunks Relacionados à Morte de Cada Escritor

In [ ]:
def buscar_chunks_relevantes(query: str, escritor_filtro: str, top_k: int = 3) -> list:
    """
    Busca os chunks mais relevantes para uma query, filtrando por escritor.

    Args:
        query: A consulta de busca
        escritor_filtro: Nome do escritor para filtrar (formato título, não wiki)
        top_k: Número de chunks a retornar

    Returns:
        Lista de dicionários com chunk, similaridade e metadados
    """
    # Gera embedding da query
    emb_query = get_embedding(client, query)

    # Filtra chunks do escritor específico e calcula similaridade
    resultados = []
    for i, (chunk, meta) in enumerate(zip(todos_chunks, metadados_escritores)):
        if meta["escritor"] == escritor_filtro:
            sim = cosine_similarity(emb_query, embeddings_escritores[i])
            resultados.append(
                {
                    "chunk": chunk,
                    "similaridade": sim,
                    "escritor": meta["escritor"],
                    "url": meta["url"],
                }
            )

    # Ordena por similaridade
    resultados.sort(key=lambda x: x["similaridade"], reverse=True)

    return resultados[:top_k]


print("✅ Função de busca semântica definida!")

In [ ]:
# Para cada escritor, buscar chunks relacionados à morte
TOP_K = 3  # Número de chunks por escritor

print("🔍 Buscando chunks relacionados à morte de cada escritor...\n")

chunks_morte_por_escritor = {}
query_morte = "morte falecimento causa óbito doença acidente assassinato"

escritores_unicos = list(set([m["escritor"] for m in metadados_escritores]))

for escritor in tqdm(escritores_unicos, desc="Buscando chunks"):
    chunks_relevantes = buscar_chunks_relevantes(
        query=query_morte, escritor_filtro=escritor, top_k=TOP_K
    )
    chunks_morte_por_escritor[escritor] = chunks_relevantes

print(
    f"\n✅ Chunks de morte encontrados para {len(chunks_morte_por_escritor)} escritores!"
)

In [ ]:
# Visualizando um exemplo
exemplo = list(chunks_morte_por_escritor.items())[0]
print(f"📖 Exemplo: {exemplo[0]}")
print(f"{'─' * 70}")
for i, chunk_info in enumerate(exemplo[1], 1):
    print(f"\n🔹 Chunk {i} (similaridade: {chunk_info['similaridade']:.4f}):")
    print(f"   {chunk_info['chunk'][:300]}...")

## 6. Definição das Classes de Causa de Morte

In [ ]:
# Classes pré-definidas para classificação
CLASSES_CAUSA_MORTE = [
    "problema_cardiaco",  # Infarto, insuficiência cardíaca, etc.
    "cancer",  # Qualquer tipo de câncer
    "acidente",  # Acidentes de trânsito, quedas, etc.
    "assassinato",  # Morte violenta intencional
    "doenca_respiratoria",  # Pneumonia, tuberculose, etc.
    "avc",  # Acidente vascular cerebral
    "diabetes",  # Complicações de diabetes
    "suicidio",  # Suicídio
    "doenca_neurologica",  # Alzheimer, Parkinson, etc.
    "complicacoes_cirurgicas",  # Morte durante ou após cirurgias
    "infeccao",  # Sepse, infecções graves
    "idade_avancada",  # Morte natural por idade
    "outra_doenca",  # Outras doenças não especificadas
    "desconhecida",  # Informação não disponível
]

CLASSES_DESCRICAO = """
Classes disponíveis para classificação:
- problema_cardiaco: Infarto, insuficiência cardíaca, arritmia, etc.
- cancer: Qualquer tipo de câncer ou neoplasia
- acidente: Acidentes de trânsito, quedas, afogamento, etc.
- assassinato: Morte violenta intencional, homicídio
- doenca_respiratoria: Pneumonia, tuberculose, enfisema, etc.
- avc: Acidente vascular cerebral (derrame)
- diabetes: Complicações de diabetes
- suicidio: Suicídio confirmado
- doenca_neurologica: Alzheimer, Parkinson, ELA, etc.
- complicacoes_cirurgicas: Morte durante ou após procedimentos cirúrgicos
- infeccao: Sepse, infecções graves
- idade_avancada: Morte natural por senilidade
- outra_doenca: Outras doenças não listadas acima
- desconhecida: Quando a informação não está disponível no contexto
"""

print("📋 Classes de causa de morte definidas:")
for classe in CLASSES_CAUSA_MORTE:
    print(f"   • {classe}")

## 7. Prompts para Classificação

### 7.1 Prompt Básico
Um prompt simples e direto, sem técnicas avançadas de engenharia de prompts.

### 7.2 Prompt com Engenharia de Prompts
Utiliza diversas técnicas:
- **Chain-of-Thought (CoT)**: Solicita raciocínio passo a passo
- **Few-shot examples**: Fornece exemplos de classificação
- **Role prompting**: Define um papel específico para o modelo
- **Output formatting**: Estrutura clara para a resposta
- **Confidence scoring**: Solicita nível de confiança

In [ ]:
# ============================================
# PROMPT BÁSICO
# ============================================


def criar_prompt_basico(escritor: str, chunks: list) -> tuple:
    """
    Cria um prompt básico para classificação de causa de morte.

    Returns:
        Tupla (system_prompt, user_prompt)
    """
    contexto = "\n\n".join([c["chunk"] for c in chunks])

    system_prompt = "Você é um assistente que classifica causas de morte."

    user_prompt = f"""Com base no texto abaixo sobre {escritor}, classifique a causa da morte.

Texto:
{contexto}

Classes possíveis: {', '.join(CLASSES_CAUSA_MORTE)}

Responda apenas com a classe e a data da morte no formato:
CLASSE: [classe]
DATA_MORTE: [data ou "desconhecida"]
"""

    return system_prompt, user_prompt


print("✅ Função de prompt básico definida!")

In [ ]:
# ============================================
# PROMPT COM ENGENHARIA DE PROMPTS AVANÇADA
# ============================================


def criar_prompt_avancado(escritor: str, chunks: list) -> tuple:
    """
    Cria um prompt avançado usando técnicas de engenharia de prompts:
    - Role prompting
    - Chain-of-Thought (CoT)
    - Few-shot examples
    - Structured output
    - Confidence scoring

    Returns:
        Tupla (system_prompt, user_prompt)
    """
    contexto = "\n\n".join([c["chunk"] for c in chunks])

    system_prompt = """Você é um especialista em história da literatura brasileira e análise biográfica, com vasto conhecimento sobre a vida e morte de escritores brasileiros. Sua tarefa é analisar cuidadosamente textos biográficos para determinar a causa de morte de escritores.

REGRAS IMPORTANTES:
1. Analise APENAS as informações presentes no contexto fornecido
2. Se não houver informação clara sobre a causa da morte, classifique como "desconhecida"
3. Seja preciso e baseie-se em evidências do texto
4. Considere termos médicos e suas equivalências populares
5. Se houver múltiplas causas mencionadas, escolha a principal/determinante"""

    user_prompt = f"""# TAREFA: Classificar a causa de morte do escritor "{escritor}"

## CONTEXTO BIOGRÁFICO:
{contexto}

## CLASSES DISPONÍVEIS:
{CLASSES_DESCRICAO}

## EXEMPLOS DE CLASSIFICAÇÃO:

Exemplo 1:
- Texto: "Morreu em decorrência de um infarto fulminante em sua residência"
- Análise: O texto menciona explicitamente "infarto", que é uma condição cardíaca
- Classificação: problema_cardiaco
- Confiança: ALTA

Exemplo 2:
- Texto: "Faleceu após longa batalha contra um tumor no pulmão"
- Análise: "Tumor" indica neoplasia/câncer
- Classificação: cancer
- Confiança: ALTA

Exemplo 3:
- Texto: "O escritor morreu em 1950, deixando grande legado literário"
- Análise: Não há menção à causa específica da morte
- Classificação: desconhecida
- Confiança: ALTA (para a classificação como desconhecida)

## SUA ANÁLISE:
Siga este processo passo a passo:

1. **EVIDÊNCIAS**: Liste as palavras-chave ou frases do texto que mencionam a morte ou causas
2. **RACIOCÍNIO**: Explique qual classe melhor se encaixa nas evidências encontradas
3. **DATA**: Identifique a data da morte, se mencionada
4. **CLASSIFICAÇÃO FINAL**: Forneça sua resposta no formato estruturado abaixo

## RESPOSTA (formato obrigatório):
EVIDENCIAS: [liste as evidências encontradas no texto]
RACIOCINIO: [explique seu raciocínio]
CLASSE: [uma das classes listadas]
DATA_MORTE: [data no formato "DD de mês de AAAA" ou "desconhecida"]
CONFIANCA: [ALTA, MEDIA ou BAIXA]
"""

    return system_prompt, user_prompt


print("✅ Função de prompt avançado definida!")

In [ ]:
# Visualizando a diferença entre os prompts
exemplo_escritor = list(chunks_morte_por_escritor.keys())[0]
exemplo_chunks = chunks_morte_por_escritor[exemplo_escritor]

print("=" * 80)
print("📝 COMPARAÇÃO DOS PROMPTS")
print("=" * 80)

sys_basico, user_basico = criar_prompt_basico(exemplo_escritor, exemplo_chunks)
sys_avancado, user_avancado = criar_prompt_avancado(exemplo_escritor, exemplo_chunks)

print(f"\n🔹 PROMPT BÁSICO:")
print(f"   System prompt: {len(sys_basico)} caracteres")
print(f"   User prompt: {len(user_basico)} caracteres")

print(f"\n🔹 PROMPT AVANÇADO:")
print(f"   System prompt: {len(sys_avancado)} caracteres")
print(f"   User prompt: {len(user_avancado)} caracteres")

print(
    f"\n📊 O prompt avançado é {len(user_avancado)/len(user_basico):.1f}x maior que o básico"
)

## 8. Classificação com a API da OpenAI

In [ ]:
def classificar_causa_morte(
    escritor: str,
    chunks: list,
    usar_prompt_avancado: bool = False,
    modelo: str = "gpt-4o-mini",
) -> dict:
    """
    Classifica a causa de morte de um escritor usando a API da OpenAI.

    Args:
        escritor: Nome do escritor
        chunks: Lista de chunks relevantes
        usar_prompt_avancado: Se True, usa o prompt com técnicas de engenharia
        modelo: Modelo da OpenAI a utilizar

    Returns:
        Dicionário com a classificação e metadados
    """
    # Seleciona o tipo de prompt
    if usar_prompt_avancado:
        system_prompt, user_prompt = criar_prompt_avancado(escritor, chunks)
    else:
        system_prompt, user_prompt = criar_prompt_basico(escritor, chunks)

    try:
        response = client.chat.completions.create(
            model=modelo,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.1,  # Baixa temperatura para respostas mais consistentes
            max_tokens=500,
        )

        resposta_texto = response.choices[0].message.content

        # Extrai informações da resposta
        resultado = {
            "escritor": escritor,
            "resposta_completa": resposta_texto,
            "chunks_utilizados": [c["chunk"] for c in chunks],
            "similaridades": [c["similaridade"] for c in chunks],
            "modelo": modelo,
            "tipo_prompt": "avancado" if usar_prompt_avancado else "basico",
        }

        # Parse da resposta para extrair classe e data
        linhas = resposta_texto.upper().split("\n")
        for linha in linhas:
            if "CLASSE:" in linha:
                classe = linha.split("CLASSE:")[-1].strip().lower()
                # Limpa a classe
                classe = classe.replace("[", "").replace("]", "").strip()
                resultado["classe"] = (
                    classe if classe in CLASSES_CAUSA_MORTE else "desconhecida"
                )
            if "DATA_MORTE:" in linha or "DATA:" in linha:
                data = linha.split(":")[-1].strip()
                resultado["data_morte"] = data.lower().replace("[", "").replace("]", "")
            if "CONFIANCA:" in linha or "CONFIANÇA:" in linha:
                resultado["confianca"] = linha.split(":")[-1].strip()

        # Valores padrão se não encontrados
        if "classe" not in resultado:
            resultado["classe"] = "desconhecida"
        if "data_morte" not in resultado:
            resultado["data_morte"] = "desconhecida"
        if "confianca" not in resultado:
            resultado["confianca"] = "N/A"

        return resultado

    except Exception as e:
        return {
            "escritor": escritor,
            "erro": str(e),
            "classe": "erro",
            "data_morte": "erro",
            "chunks_utilizados": [c["chunk"] for c in chunks],
            "tipo_prompt": "avancado" if usar_prompt_avancado else "basico",
        }


print("✅ Função de classificação definida!")

In [ ]:
# Executando classificação com PROMPT BÁSICO
print("🔄 Executando classificação com PROMPT BÁSICO...\n")

resultados_basico = []

for escritor, chunks in tqdm(chunks_morte_por_escritor.items(), desc="Prompt Básico"):
    resultado = classificar_causa_morte(escritor, chunks, usar_prompt_avancado=False)
    resultados_basico.append(resultado)
    time.sleep(0.5)  # Delay para respeitar rate limits

print(f"\n✅ {len(resultados_basico)} classificações concluídas com prompt básico!")

In [ ]:
# Executando classificação com PROMPT AVANÇADO
print("🔄 Executando classificação com PROMPT AVANÇADO...\n")

resultados_avancado = []

for escritor, chunks in tqdm(chunks_morte_por_escritor.items(), desc="Prompt Avançado"):
    resultado = classificar_causa_morte(escritor, chunks, usar_prompt_avancado=True)
    resultados_avancado.append(resultado)
    time.sleep(0.5)  # Delay para respeitar rate limits

print(f"\n✅ {len(resultados_avancado)} classificações concluídas com prompt avançado!")

## 9. Criação dos DataFrames com Resultados

In [ ]:
# Criando DataFrame para PROMPT BÁSICO
df_basico = pd.DataFrame(
    [
        {
            "escritor": r["escritor"],
            "data_morte": r.get("data_morte", "desconhecida"),
            "classificacao_causa": r.get("classe", "desconhecida"),
            "chunk_1": (
                r["chunks_utilizados"][0] if len(r["chunks_utilizados"]) > 0 else ""
            ),
            "chunk_2": (
                r["chunks_utilizados"][1] if len(r["chunks_utilizados"]) > 1 else ""
            ),
            "chunk_3": (
                r["chunks_utilizados"][2] if len(r["chunks_utilizados"]) > 2 else ""
            ),
            "resposta_modelo": r.get("resposta_completa", ""),
        }
        for r in resultados_basico
    ]
)

print("📊 DataFrame com resultados do PROMPT BÁSICO:")
print(f"   Shape: {df_basico.shape}")
display(df_basico[["escritor", "data_morte", "classificacao_causa"]].head(10))

In [ ]:
# Criando DataFrame para PROMPT AVANÇADO
df_avancado = pd.DataFrame(
    [
        {
            "escritor": r["escritor"],
            "data_morte": r.get("data_morte", "desconhecida"),
            "classificacao_causa": r.get("classe", "desconhecida"),
            "confianca": r.get("confianca", "N/A"),
            "chunk_1": (
                r["chunks_utilizados"][0] if len(r["chunks_utilizados"]) > 0 else ""
            ),
            "chunk_2": (
                r["chunks_utilizados"][1] if len(r["chunks_utilizados"]) > 1 else ""
            ),
            "chunk_3": (
                r["chunks_utilizados"][2] if len(r["chunks_utilizados"]) > 2 else ""
            ),
            "resposta_modelo": r.get("resposta_completa", ""),
        }
        for r in resultados_avancado
    ]
)

print("📊 DataFrame com resultados do PROMPT AVANÇADO:")
print(f"   Shape: {df_avancado.shape}")
display(
    df_avancado[["escritor", "data_morte", "classificacao_causa", "confianca"]].head(10)
)

In [ ]:
# Salvando os DataFrames em CSV
df_basico.to_csv(
    "classificacao_morte_prompt_basico.csv", index=False, encoding="utf-8-sig"
)
df_avancado.to_csv(
    "classificacao_morte_prompt_avancado.csv", index=False, encoding="utf-8-sig"
)

print("💾 DataFrames salvos:")
print("   - classificacao_morte_prompt_basico.csv")
print("   - classificacao_morte_prompt_avancado.csv")

## 10. Análise Comparativa dos Resultados

In [ ]:
# Comparação das classificações entre os dois prompts
print("=" * 80)
print("📊 COMPARAÇÃO: PROMPT BÁSICO vs PROMPT AVANÇADO")
print("=" * 80)

# Juntando os resultados para comparação
df_comparacao = pd.DataFrame(
    {
        "escritor": df_basico["escritor"],
        "classe_basico": df_basico["classificacao_causa"],
        "classe_avancado": df_avancado["classificacao_causa"],
        "confianca_avancado": df_avancado["confianca"],
    }
)

# Calculando concordância
df_comparacao["concordancia"] = (
    df_comparacao["classe_basico"] == df_comparacao["classe_avancado"]
)
concordancia_total = df_comparacao["concordancia"].mean() * 100

print(f"\n🎯 Taxa de concordância entre prompts: {concordancia_total:.1f}%")
print(f"\n📋 Casos onde os prompts discordaram:")
discordantes = df_comparacao[~df_comparacao["concordancia"]]
if len(discordantes) > 0:
    display(discordantes)
else:
    print("   Todos os casos concordaram!")

In [ ]:
# Distribuição das classes - Prompt Básico
print("\n📊 Distribuição de classes - PROMPT BÁSICO:")
distribuicao_basico = df_basico["classificacao_causa"].value_counts()
print(distribuicao_basico)

print("\n📊 Distribuição de classes - PROMPT AVANÇADO:")
distribuicao_avancado = df_avancado["classificacao_causa"].value_counts()
print(distribuicao_avancado)

In [ ]:
# Visualização gráfica
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico 1: Prompt Básico
distribuicao_basico.plot(kind="barh", ax=axes[0], color="steelblue")
axes[0].set_title("Distribuição - Prompt Básico", fontsize=14)
axes[0].set_xlabel("Quantidade")
axes[0].set_ylabel("Causa da Morte")

# Gráfico 2: Prompt Avançado
distribuicao_avancado.plot(kind="barh", ax=axes[1], color="darkgreen")
axes[1].set_title("Distribuição - Prompt Avançado", fontsize=14)
axes[1].set_xlabel("Quantidade")
axes[1].set_ylabel("Causa da Morte")

plt.tight_layout()
plt.savefig("comparacao_distribuicao_causas.png", dpi=150, bbox_inches="tight")
plt.show()

print("📈 Gráfico salvo: comparacao_distribuicao_causas.png")

In [ ]:
# Tabela final resumida
print("\n" + "=" * 100)
print("📋 TABELA FINAL: CLASSIFICAÇÃO DE CAUSA DE MORTE DOS ESCRITORES BRASILEIROS")
print("=" * 100)

tabela_final = pd.DataFrame(
    {
        "Escritor": df_basico["escritor"],
        "Data Morte": df_avancado["data_morte"],
        "Causa (Básico)": df_basico["classificacao_causa"],
        "Causa (Avançado)": df_avancado["classificacao_causa"],
        "Confiança": df_avancado["confianca"],
    }
)

# Formatando para exibição
pd.set_option("display.max_colwidth", 50)
pd.set_option("display.max_rows", 35)

display(tabela_final)

In [ ]:
# Salvando tabela final
tabela_final.to_csv("tabela_final_classificacao.csv", index=False, encoding="utf-8-sig")
print("💾 Tabela final salva: tabela_final_classificacao.csv")

## 11. Exemplo de Resposta Detalhada do Prompt Avançado

In [ ]:
# Mostrando um exemplo completo de resposta do prompt avançado
exemplo_idx = 0
exemplo_resultado = resultados_avancado[exemplo_idx]

print("=" * 80)
print(f"📖 EXEMPLO DETALHADO: {exemplo_resultado['escritor']}")
print("=" * 80)

print("\n📝 CHUNKS UTILIZADOS:")
for i, chunk in enumerate(exemplo_resultado["chunks_utilizados"], 1):
    print(f"\n--- Chunk {i} ---")
    print(chunk[:500] + "..." if len(chunk) > 500 else chunk)

print("\n" + "=" * 80)
print("🤖 RESPOSTA DO MODELO (Prompt Avançado):")
print("=" * 80)
print(exemplo_resultado.get("resposta_completa", "Não disponível"))

## 📌 Conclusões

Este notebook demonstrou um pipeline completo para:

1. **Coleta automatizada** de dados biográficos de 30 escritores brasileiros da Wikipedia
2. **Processamento com embeddings** para busca semântica eficiente
3. **Classificação com LLM** usando duas estratégias de prompt

### Diferenças entre os prompts:

| Aspecto | Prompt Básico | Prompt Avançado |
|---------|---------------|-----------------|
| Tamanho | ~200 tokens | ~800 tokens |
| Técnicas | Instrução direta | CoT, Few-shot, Role |
| Estrutura | Resposta livre | Formato estruturado |
| Confiança | Não solicita | Inclui score |
| Custo API | Menor | Maior |

### Recomendações:

- **Use o prompt básico** quando: custo é prioritário, contexto é claro, alta velocidade é necessária
- **Use o prompt avançado** quando: precisão é crítica, casos ambíguos, precisa de explicabilidade